In [ ]:
import sys
sys.path.append("..") #this is to add the avobe folder to the package directory
import geopandas as gpd
import rasterio
from rasterio.plot import show
import nexustool.weap_tools as wp
import os
import matplotlib.pyplot as plt
from nexustool.gis_tools import download_data
import fiona
fiona.drvsupport.supported_drivers['kml'] = 'rw' # enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw' # enable KML support which is disabled by default

## Reading files

In [ ]:
gis_folder = os.path.join('data', 'gis')
sch_folder = os.path.join('data', 'schematic')

provinces = gpd.read_file(os.path.join(gis_folder, 'admin', 'provinces.gpkg'), encoding='utf-8')
groundwater = gpd.read_file(os.path.join(sch_folder, 'Groundwater.kml'), encoding='utf-8')
wwtp = gpd.read_file(os.path.join(sch_folder, 'Wastewater Treatment Plants.kml'), encoding='utf-8')
other_supply = gpd.read_file(os.path.join(sch_folder, 'Other Supplies.kml'), encoding='utf-8')
river_withdrawals = gpd.read_file(os.path.join(sch_folder, 'River Withdrawals.kml'), encoding='utf-8')
demand_sites = gpd.read_file(os.path.join(sch_folder, 'Demand Sites.kml'), encoding='utf-8')
catchments = gpd.read_file(os.path.join(sch_folder, 'Catchments.kml'), encoding='utf-8')
diversion = gpd.read_file(os.path.join(sch_folder, 'Diversions.kml'), encoding='utf-8')
reservoirs = gpd.read_file(os.path.join(sch_folder, 'Reservoirs.kml'), encoding='utf-8')
links = gpd.read_file(os.path.join(sch_folder, 'Transmission Links.kml'), encoding='utf-8')

## Downloading the GIS data

In [ ]:
url = 'http://viewfinderpanoramas.org/dem3/I29.zip'
file_path = os.path.join('data', 'gis', 'dem', 'I29.zip')
download_data(url, file_path)

url = 'http://viewfinderpanoramas.org/dem3/H29.zip'
file_path = os.path.join('data', 'gis', 'dem', 'H29.zip')
download_data(url, file_path)

url = 'https://souss-massa-dev.s3.us-east-2.amazonaws.com/post_build/Africa_model_wtd_v2.nc'
file_path = os.path.join('data', 'gis', 'wtd', 'Africa_model_wtd_v2.nc')
download_data(url, file_path)

## Converting geometries and dropping unecesary columns

In [ ]:
for gdf in [groundwater, wwtp, other_supply, river_withdrawals,
            demand_sites, catchments, diversion, reservoirs, links]:
    gdf['geometry'] = wp.polyz_to_poly(gdf)
    gdf.drop(gdf.columns.difference(['Name','geometry']), 1, inplace=True)

In [ ]:
for gdf in [groundwater, wwtp, other_supply, river_withdrawals,
            reservoirs, demand_sites, catchments]:
    gdf.rename(columns={'Name': 'point'}, inplace=True)

links.rename(columns={'Name': 'links'}, inplace=True)
diversion.rename(columns={'Name': 'diversion'}, inplace=True)

## Defining types

In [ ]:
groundwater['type'] = 'Groundwater supply'
wwtp['type'] = 'WWTP'
other_supply['type'] = 'Other supply'
river_withdrawals['type'] = 'Surfacewater withdrawal'
reservoirs['type'] = 'Reservoir supply'
catchments['type'] = 'Catchment'
demand_sites['type'] = 'Demand site'
diversion['type'] = 'Transmission Pipeline'
links['type'] = 'Transmission links'

## Cleaning up the data

In [ ]:
supply_points = groundwater.append([other_supply, reservoirs, river_withdrawals, wwtp], ignore_index=True, sort=False)
demand_points = demand_sites.append(catchments, ignore_index=True, sort=False)
links['links'] = links.links.str.replace('Groundwater','GW')
links['links'] = links.links.str.replace('Grounwater','GW')
links['links'] = links.links.str.replace('GW of ','')
links['links'] = links.links.str.replace('GW ','')
links['links'] = links.links.str.strip()
links['links'] = links.links.str.replace(' El Guerdane',' I_El Guerdane')

## Spatially joining some layers

In [ ]:
demand_links = gpd.sjoin(demand_points, links, how='inner', op='intersects')
supply_links = gpd.sjoin(supply_points, links, how='inner', op='intersects')

In [ ]:
demand_links.rename(columns={'type_left': 'type'}, inplace=True)
demand_links.drop(columns=['type_right', 'index_right'], inplace=True)
supply_links.rename(columns={'type_left': 'type'}, inplace=True)
supply_links.drop(columns=['type_right', 'index_right'], inplace=True)

In [ ]:
dff = gpd.sjoin(demand_links, provinces[['Province','geometry']], how='inner', op='within')
dff.drop(columns=['index_right'], inplace=True)
df_temp = demand_links.loc[~demand_links.point.isin(dff.point.unique())].copy()
df_temp['Province'] = 'Tiznit'
demand_links = dff.append(df_temp, sort=False)

In [ ]:
dff = gpd.sjoin(supply_links, provinces[['Province','geometry']], how='inner', op='within')
dff.drop(columns=['index_right'], inplace=True)
df_temp = supply_links.loc[~supply_links.point.isin(dff.point.unique())].copy()
supply_links = dff.append(df_temp, sort=False)
df_temp['Province'] = 'Chtouka-Aït Baha'
supply_links = dff.append(df_temp, sort=False)

## Displaying the system

In [ ]:
base = provinces.plot(color='white', edgecolor='black', figsize=(12, 12))
data = demand_links.append([supply_links, wwtp, diversion, links], ignore_index=True, sort=False)
data.plot(ax=base, column='type', cmap='Spectral_r', legend=True)

## Masking and displaying rasters

In [ ]:
out_image, out_meta = wp.mask_raster(os.path.join(gis_folder, 'wtd', 'Africa_model_wtd_v2.nc'), 
                                  os.path.join(gis_folder, 'admin', 'provinces.gpkg'), 'EPSG:4326')
       
with rasterio.open(os.path.join(gis_folder, 'wtd', 'Souss-Massa WTD.tif'), 'w', **out_meta) as dest:
    dest.write(out_image)

In [ ]:
with rasterio.open(os.path.join(gis_folder, 'wtd', 'Souss-Massa WTD.tif')) as src:
    fig, ax = plt.subplots(figsize=(12, 12))
    show(src, ax=ax, aspect='auto', cmap='gist_earth')

In [ ]:
dem_path = os.path.join(gis_folder, 'dem', 'Souss-Massa DEM.tif')
if not os.path.exists(dem_path):
    wp.merge_rasters(os.path.join(gis_folder, 'dem', '*', '*.hgt'), 
                     'EPSG:4326', 
                     dem_path)

with rasterio.open(os.path.join(gis_folder, 'dem', 'Souss-Massa DEM.tif')) as src:
    fig, ax = plt.subplots(figsize=(12, 12))
    show(src, ax=ax, aspect='auto', cmap='terrain')

## Sampling raster data

In [ ]:
groundwater['wtd_m'] = wp.sample_raster(os.path.join(gis_folder, 'wtd', 'Africa_model_wtd_v2.nc'), 
                                        groundwater)
groundwater.loc[groundwater.point=='Souss GW','wtd_m'] = 170
groundwater.loc[groundwater.point=='Chtouka GW','wtd_m'] = 85

In [ ]:
supply_links['elevation'] = wp.sample_raster(dem_path, supply_links)
demand_links['elevation'] = wp.sample_raster(dem_path, demand_links)
wwtp['elevation'] = wp.sample_raster(dem_path, wwtp)

## Spatially joining diversions with reservoirs and river withdrawals

In [ ]:
diversions = gpd.sjoin(diversion, reservoirs, how='inner', op='intersects')
diversions.drop(columns=['index_right'], inplace=True)
diversions.rename(columns={'type_right': 'type_supply'}, inplace=True)
diversions = gpd.sjoin(diversions, river_withdrawals, how='inner', op='intersects')
diversions.drop(columns=['index_right'], inplace=True)
diversions.rename(columns={'point_left': 'Supply point', 'point_right': 'Demand point',
                           'type_left': 'type', 'type': 'type_demand'}, inplace=True)

## Sampling elevation data

In [ ]:
dff1 = river_withdrawals.loc[river_withdrawals.point.isin(diversions['Demand point'])].copy()
dff1['diversion'] = dff1.point.map(diversions.set_index('Demand point').diversion)
dff1['elevation'] = wp.sample_raster(dem_path, dff1)

In [ ]:
dff2 = reservoirs.loc[reservoirs.point.isin(diversions['Supply point'])].copy()
dff2['diversion'] = dff2.point.map(diversions.groupby('Supply point').agg({'diversion': 'first'})['diversion'])
dff2['elevation'] = wp.sample_raster(dem_path, dff2)
MerchidSudMoroc = 26192
dff2.crs = 4326
dff2.to_crs(f'epsg:{MerchidSudMoroc}', inplace=True)

## Calculating pipelines distance

In [ ]:
dff1.crs = 4326
dff1.to_crs(f'epsg:{MerchidSudMoroc}', inplace=True)
dff1.set_index('diversion', inplace=True)
dff1['distance'] = dff1.distance(dff2.set_index('diversion')).copy()

## Calculating elevation defference

In [ ]:
diversions['distance'] = diversions['Demand point'].map(dff1.set_index('point')['distance'])
diversions['elevation_diff'] = diversions['Demand point'].map(dff1.set_index('point')['elevation']) - \
                               diversions['Supply point'].map(dff2.set_index('point')['elevation'])

In [ ]:
all_points = supply_links.append(demand_links, sort=False, ignore_index=True)
all_points.drop_duplicates(subset="point", inplace=True)

## Saving the processed layers

In [ ]:
output_folder = os.path.join('data', 'gis', 'processed layers')
os.makedirs(output_folder, exist_ok = True)

demand_links.to_file(os.path.join(output_folder, 'demand_points.gpkg'), driver="GPKG", OVERWRITE='YES')
supply_links.to_file(os.path.join(output_folder, 'supply_points.gpkg'), driver="GPKG", OVERWRITE='YES')
wwtp.to_file(os.path.join(output_folder, 'wwtp.gpkg'), driver="GPKG", OVERWRITE='YES')
diversions.to_file(os.path.join(output_folder, 'pipelines.gpkg'), driver="GPKG", OVERWRITE='YES')
groundwater.to_file(os.path.join(output_folder, 'groundwater.gpkg'), driver="GPKG", OVERWRITE='YES')
all_points.to_file(os.path.join(output_folder, 'all_points.gpkg'), driver="GPKG", OVERWRITE='YES')